In [20]:
'''
Author: xushengyichn 54436848+xushengyichn@users.noreply.github.com
Date: 2023-01-24 14:28:48
LastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com
LastEditTime: 2023-01-25 18:51:17
FilePath: \20230124优化问题\bayesian.ipynb
Description: 创建optuna优化，优化模态1，寻找最优TMD参数

Copyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. 
'''


'\nAuthor: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nDate: 2023-01-24 14:28:48\nLastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nLastEditTime: 2023-01-25 18:51:17\nFilePath: \x8230124优化问题\x08ayesian.ipynb\nDescription: 创建optuna优化，优化模态1，寻找最优TMD参数\n\nCopyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. \n'

In [21]:
#%% 1. 导入库
import logging
import sys
import time
import matlab
import matlab.engine
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score,KFold,cross_validate
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
import scipy.io

In [22]:
def objective(mu,xi,alpha,init_points,n_iter):
    eng = matlab.engine.start_matlab()
    def black_box_function(fTMD1,dTMD1,xTMD1):
        total_tmd_mass_ratio = 0.02 # 总质量比 The total mass ratio
        mass_six_span = 10007779.7 # 深中通道非通航桥六跨连续梁质量 The mass of 6-span continuous beam of the non-navigational bridge of the Zhenzhong-Link
        total_tmd_mass = total_tmd_mass_ratio * mass_six_span # 总质量 The total mass
        
        mTMD1=total_tmd_mass/6 # 质量 The mass mTMD1
        fTMD1=0.7+fTMD1*0.3 # 频率 The frequency fTMD1
        dTMD1=0.05+dTMD1*0.15 # 阻尼比 The damping ratio dTMD1
        xTMD1=xTMD1*660 # TMD1的x坐标 The x-coordinate of TMD1
        t_length=matlab.double(100) # 时间长度 The time length
        number_of_modes_to_control=matlab.double([1]) # 控制模态 The controlled modes
        number_of_modes_to_consider=10 # 考虑模态 The considered modes
        number_of_tmds=1 # TMD数量 The number of TMDs
        modal_damping_ratios=np.ones((1,number_of_modes_to_consider))*0.003 # 模态阻尼比 The modal damping ratios
        
        result = -eng.c_mode1(number_of_modes_to_control,number_of_modes_to_consider,number_of_tmds,modal_damping_ratios,t_length,mTMD1,fTMD1,dTMD1,xTMD1)
        return result
    #%% 3. 定义优化器
    optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds={"fTMD1": (0,1),
                "dTMD1": (0,1),
                "xTMD1": (0,1),
                },
        verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
        random_state=1,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        xi=xi,
        # What follows are GP regressor parameters
        kernel=Matern(nu=mu),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=alpha,
        normalize_y=True,
        n_restarts_optimizer=5,
    )
    
    loss=optimizer.max['target']
    return loss

    
    
    

In [23]:
#%% 2. 调用函数

def obj(trial):
    mu_ = trial.suggest_float('mu_', 0.0,2.0,step=1.0)
    kappa = trial.suggest_float('xi', 1e-4, 1e-1,log=True)
    alpha = trial.suggest_float('alpha', 1e-10, 1e-2,log=True)
    init_points = 10
    # init_points = trial.suggest_int('init_points', 10, 100)
    n_iter = 100
    # n_iter = trial.suggest_int('n_iter', 10, 100)
    if mu_==0:
        mu=0.5
    elif mu_==1:
        mu=1.5
    else:
        mu=2.5
    loss = objective(mu,kappa,alpha,init_points,n_iter)
    return loss


optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "opt_mode1"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(direction='maximize',study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(obj, n_trials=50)

print(study.best_params)
print(study.best_value)

[I 2023-01-30 15:42:06,754] Using an existing study with name 'opt_mode1' instead of creating a new one.


Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
|   iter    |  target   |   dTMD1   |   fTMD1   |   xTMD1   |
-------------------------------------------------------------
| 1         | -0.2102   | 0.417     | 0.7203    | 0.0001144 |
| 2         | -0.1655   | 0.3023    | 0.1468    | 0.09234   |
| 3         | -0.08284  | 0.1863    | 0.3456    | 0.3968    |
| 4         | -0.1988   | 0.5388    | 0.4192    | 0.6852    |
| 5         | -0.204    | 0.2045    | 0.8781    | 0.02739   |
| 6         | -0.1085   | 0.6705    | 0.4173    | 0.5587    |
| 7         | -0.1944   | 0.1404    | 0.19

[I 2023-01-30 15:44:50,182] Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.


Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.
Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.
Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.
Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.
Trial 8 finished with value: -0.037566359274518596 and parameters: {'mu_': 1.0, 'xi': 0.002215464801609677, 'alpha': 0.0018795808332718609}. Best is trial 5 with value: -0.033562892473544055.
Trial 8 finished with value: -0.03756635

[I 2023-01-30 15:47:28,278] Trial 9 finished with value: -0.036764160789545204 and parameters: {'mu_': 1.0, 'xi': 0.000556419184715521, 'alpha': 0.00035008801635406776}. Best is trial 5 with value: -0.033562892473544055.


...

Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alpha': 1.799610031726564e-06}. Best is trial 32 with value: 0.0.
Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alpha': 1.799610031726564e-06}. Best is trial 32 with value: 0.0.
Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alpha': 1.799610031726564e-06}. Best is trial 32 with value: 0.0.
Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alpha': 1.799610031726564e-06}. Best is trial 32 with value: 0.0.
Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alpha': 1.799610031726564e-06}. Best is trial 32 with value: 0.0.
Trial 55 finished with value: -0.03542667091688735 and parameters: {'mu_': 0.0, 'xi': 0.00015040390714384156, 'alph

[I 2023-01-30 19:11:40,208] Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.


Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.
Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.
Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.
Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.
Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha': 1.0221847531302673e-08}. Best is trial 32 with value: 0.0.
Trial 56 finished with value: -0.03609709123913123 and parameters: {'mu_': 1.0, 'xi': 0.0006198783895672973, 'alpha

[I 2023-01-30 19:14:26,877] Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.


Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.
Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.
Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.
Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.
Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha': 1.4619789806232488e-05}. Best is trial 32 with value: 0.0.
Trial 57 finished with value: -0.03599670695862916 and parameters: {'mu_': 0.0, 'xi': 0.0023059666731758545, 'alpha

In [25]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "opt_mode1"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(direction='maximize',study_name=study_name, storage=storage_name, load_if_exists=True)
print(study.best_params)
print(study.best_value)
# print(study.trials)

print(study.directions)


[I 2023-01-30 19:15:24,511] Using an existing study with name 'opt_mode1' instead of creating a new one.


Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
Using an existing study with name 'opt_mode1' instead of creating a new one.
{'alpha': 8.960456511761172e-06, 'mu_': 1.0, 'xi': 0.0012265019946638392}
0.0
[<StudyDirection.MAXIMIZE: 2>]
